## Creating transformed dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Behavioral_Risk_Factor_Surveillance_System__BRFSS__Prevalence_Data__2011_to_present_.csv', sep = ";")

In [ ]:
df['Sample_Size'] = df['Sample_Size'].str.replace(",", "")
df['Sample_Size'] = df['Sample_Size'].astype('int64')
#Cahnge columns name
df.rename(columns = {"Break_Out":"Category",  "Break_Out_Category":"Class_Category"}, inplace= True)


In [ ]:
# Change the response to binary answers
df['Response'] = df['Response'].replace({"Never smoked": "No", "Former smoker":"No", "Smoke some days":"Yes", "Smoke everyday":"Yes"})


In [ ]:
# Creating dictionary as a reference
total_sample_size = dict(df.groupby(["Year","Question","Category"])["Sample_Size"].sum())

def get_percentage(col1,col2,col3,col4):
    for year, total_number in total_sample_size.items():
        if (col1, col2, col3) == year:
            return round((col4 / total_number)*100, 2)
            
def get_total_size(col1,col2,col3):
    for year, total_number in total_sample_size.items():
        if (col1,col2,col3) == year:
            return total_number

In [ ]:
# Vectorizing the function to numpy to shorten the process
vectfunc = np.vectorize(get_percentage)
vect_get_total_size = np.vectorize(get_total_size)

In [ ]:
# creating new column for percentage
df["Percentage"] = vectfunc(df['Year'], df['Question'], df['Category'],df['Sample_Size'])

In [ ]:
# creating new column for Total Sample Size based on the question and year
df["Total_SS"] = vect_get_total_size(df['Year'], df['Question'], df['Category'])

In [ ]:
df_final = df.groupby(["Year","Class", "Topic", "Question", "Response","Category","Class_Category", "Total_SS"]).aggregate({'Sample_Size': 'sum','Percentage': 'sum'}).reset_index()
df_final.to_csv("BRFSS.csv", index = False)


In [ ]:
df.to_csv("brfss_backup.csv", index = False)

# Creating dataset for Smoker Status with additional lattitude and longitude data

In [ ]:
import pandas as pd
import re
import numpy as np


In [ ]:
df_smokers = pd.read_csv("brfss_backup.csv")

In [ ]:
# Removing null values
df_smokers.drop(df_smokers[df_smokers["Locationabbr"] == "UW"].index, axis = 0, inplace = True)
df_smokers.drop(df_smokers[df_smokers["Locationabbr"] == "US"].index, axis = 0, inplace = True)

In [ ]:
df_smokers = df_smokers.loc[df_smokers['Topic'] == 'Smoker Status']

In [ ]:
# Creating function to seperate lat and long from Geo Location function

def get_lat(col):
    tmp = []
    for i in col:
        latlong = re.findall(r"(-?\d+.\d+)", i) 
        tmp.append(latlong[0])
    return tmp

def get_long(col):
    tmp = []
    for i in col:
        latlong = re.findall(r"(-?\d+.\d+)", i) 
        tmp.append(latlong[1])
    return tmp

df_smokers['latitude'] = get_lat(df_smokers['GeoLocation'])
df_smokers['longitude'] = get_long(df_smokers['GeoLocation'])




In [ ]:
# Make the percentage of sample size according to the location
total_sample_size = dict(df_smokers.groupby(["Year","Locationdesc"])["Sample_Size"].sum())

def get_percentage(a,b,c):
    for year, total_number in total_sample_size.items():
        if (a, b) == year:
            return round((c / total_number)*100, 2)
            
def get_total_size(a,b):
    for year, total_number in total_sample_size.items():
        if (a,b) == year:
            return total_number

get_percentage_vect = np.vectorize(get_percentage)
get_total_size_vect = np.vectorize(get_total_size)

df_smokers['Percentage_Loc'] = get_percentage_vect(df_smokers['Year'], df_smokers['Locationdesc'], df_smokers['Sample_Size'])
df_smokers['Total_SS_Loc'] = get_total_size_vect(df_smokers['Year'], df_smokers['Locationdesc'])

In [ ]:

df_smokers_to_csv = df_smokers.groupby(["Year","Locationdesc","Response","Total_SS_Loc","latitude", "longitude"]).aggregate({"Percentage_Loc":"sum", "Sample_Size":"sum"}).reset_index()


In [ ]:
df_smokers_to_csv.to_csv("brfss_smokers.csv", index = False)